In [1]:
import ads

ads.set_auth(auth="resource_principal")

In [2]:
import tensorflow as tf

# Create the TensorFlow Model
mnist = tf.keras.datasets.mnist
(trainx, trainy), (testx, testy) = mnist.load_data()
trainx, testx = trainx / 255.0, testx / 255.0

model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10),
])

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer="adam", loss=loss_fn, metrics=["accuracy"])
model.fit(trainx, trainy, epochs=1)

2025-11-29 21:52:11.052199: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-29 21:52:11.057398: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


1875/1875 [==============================] - 8s 4ms/step - loss: 0.2968 - accuracy: 0.9141


In [3]:
# Exercise/validate the prediction
model.predict(testx[0:1])

array([[-1.1906428 , -6.929907  ,  1.8112243 ,  3.0260627 , -7.516226  ,
        -1.3296056 , -9.604608  ,  8.443379  , -1.4479129 ,  0.25440818]],
      dtype=float32)

In [4]:
## TENSORFLOW FRAMEWORK SERIALIZATION
# Create an ADS model
from ads.common.model_metadata import UseCaseType
from ads.model.framework.tensorflow_model import TensorFlowModel
from uuid import uuid4

tensorflow_model = TensorFlowModel(estimator=model, artifact_dir=f"./model-artifact-{str(uuid4())}")

# Summarize the checkpoitn ADS workflow status
tensorflow_model.summary_status()

Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Available     Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Not Available Local tested .predict from score.py                               
save()    Not Available Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  Not Available Deployed the model                                                
predict() Not Available Called deployment predict endpoint

In [5]:
# Prepare the ADS Model
tensorflow_model.prepare(
    X_sample=trainx,
    y_sample=trainy,
    use_case_type=UseCaseType.MULTINOMIAL_CLASSIFICATION,
)

# Summarize the checkpoitn ADS workflow status
tensorflow_model.summary_status()

Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Done          Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Available     Local tested .predict from score.py                               
save()    Available     Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  Not Available Deployed the model                                                
predict() Not Available Called deployment predict endpoint

In [6]:
# Verify the ADS model
tensorflow_model.verify(testx)

# Summarize the checkpoitn ADS workflow status
tensorflow_model.summary_status()

Start loading model.h5 from model directory /home/datascience/demo/model-artifact-68790424-16c5-4c17-9fa3-00a780827618 ...
Model is successfully loaded.


Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Done          Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Done          Local tested .predict from score.py                               
save()    Available     Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  Not Available Deployed the model                                                
predict() Not Available Called deployment predict endpoint

In [7]:
# Save the ADS model
model_id = tensorflow_model.save()

# Summarize the checkpoitn ADS workflow status
tensorflow_model.summary_status()

Start loading model.h5 from model directory /home/datascience/demo/model-artifact-68790424-16c5-4c17-9fa3-00a780827618 ...
Model is successfully loaded.
['output_schema.json', 'score.py', 'model.h5', 'runtime.yaml']


loop1:   0%|          | 0/5 [00:00<?, ?it/s]

Actions Needed
Step      Status        Details                                                           
initiate  Done          Initiated the model                                               
prepare() Done          Generated runtime.yaml                                            
                        Generated score.py                                                
                        Serialized model                                                  
                        Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Done          Local tested .predict from score.py                               
save()    Done          Conducted Introspect Test                                         
                        Uploaded artifact to model catalog                                
deploy()  Available     Deployed the model                                                
predict() Not Available Called deployment predict endpoint

In [8]:
# Deploy and create an endpoint for the TensorFlow model
tensorflow_model.deploy(
    display_name="DIY TensorFlow Model For MNIST Classification",
    #deployment_log_group_id="ocid1.loggroup.oc1.xxx.xxxxx",
    #deployment_access_log_id="ocid1.log.oc1.xxx.xxxxx",
    #deployment_predict_log_id="ocid1.log.oc1.xxx.xxxxx",
    # Shape config details mandatory for flexible shapes:
    deployment_instance_shape="VM.Standard.E4.Flex",
    deployment_ocpus=1,
    deployment_memory_in_gbs=16,
)
print(f"Endpoint: {tensorflow_model.model_deployment.url}")

# Summarize the checkpoitn ADS workflow status
tensorflow_model.summary_status()

loop1:   0%|          | 0/6 [00:00<?, ?it/s]

Endpoint: https://modeldeployment.us-chicago-1.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.us-chicago-1.amaaaaaakoygqqiavmoypckh3zcslweisg2v4lhsiez26ynsyvnbzmvg7gdq


Actions Needed
Step      Status    Details                                                           
initiate  Done      Initiated the model                                               
prepare() Done      Generated runtime.yaml                                            
                    Generated score.py                                                
                    Serialized model                                                  
                    Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Done      Local tested .predict from score.py                               
save()    Done      Conducted Introspect Test                                         
                    Uploaded artifact to model catalog                                
deploy()  ACTIVE    Deployed the model                                                
predict() Available Called deployment predict endpoint

In [9]:
# Generate prediction by invoking the deployed endpoint
tensorflow_model.predict(testx[:3])['prediction']

[[-1.1906434297561646,
  -6.929907321929932,
  1.811224102973938,
  3.026061773300171,
  -7.516226291656494,
  -1.3296053409576416,
  -9.604606628417969,
  8.443377494812012,
  -1.447912573814392,
  0.25440841913223267],
 [-1.2026904821395874,
  -1.284335970878601,
  9.257678031921387,
  3.409111738204956,
  -13.907402038574219,
  1.2960190773010254,
  -2.35964298248291,
  -11.947013854980469,
  2.248516798019409,
  -11.872596740722656],
 [-5.145094394683838,
  5.267521858215332,
  -0.0835074931383133,
  -1.2678934335708618,
  -3.0604896545410156,
  -1.6988072395324707,
  -1.682331919670105,
  -0.45520487427711487,
  0.14755238592624664,
  -3.5285167694091797]]

In [10]:
# Summarize the checkpoitn ADS workflow status
tensorflow_model.summary_status()

Actions Needed
Step      Status    Details                                                           
initiate  Done      Initiated the model                                               
prepare() Done      Generated runtime.yaml                                            
                    Generated score.py                                                
                    Serialized model                                                  
                    Populated metadata(Custom, Taxonomy and Provenance)               
verify()  Done      Local tested .predict from score.py                               
save()    Done      Conducted Introspect Test                                         
                    Uploaded artifact to model catalog                                
deploy()  ACTIVE    Deployed the model                                                
predict() Available Called deployment predict endpoint

In [11]:
# Prepare data sample for prediction and save it to file 'data-payload'
from io import BytesIO
import base64
import numpy as np

data = testx[:3]
np_bytes = BytesIO()
np.save(np_bytes, data, allow_pickle=True)
data = base64.b64encode(np_bytes.getvalue()).decode("utf-8")
with open('data-payload', 'w') as f:
    f.write('{"data": "' + data + '", "data_type": "numpy.ndarray"}')

In [12]:
# Alternately, the OCI CLI may be invoked with the command line
print("Use the following command line to invoke the deployment via the OCI CLI...\n")
print("oci raw-request --http-method POST --target-uri " + tensorflow_model.model_deployment.url + "/predict --request-body file://data-payload")

Use the following command line to invoke the deployment via the OCI CLI...

oci raw-request --http-method POST --target-uri https://modeldeployment.us-chicago-1.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.us-chicago-1.amaaaaaakoygqqiavmoypckh3zcslweisg2v4lhsiez26ynsyvnbzmvg7gdq/predict --request-body file://data-payload
